# MDA工作流 with 自适应任务分解

这个notebook展示了如何应用自适应任务分解知识，让MDA工作流能够：
1. 定义每个步骤的后置断言
2. 失败时自动分解任务
3. 使用TODO管理跟踪进度

## 1. 初始化

In [ ]:
import os
import sys
from pathlib import Path
import json
import shutil

# 添加项目路径
sys.path.insert(0, str(Path.cwd()))

from core.react_agent import GenericReactAgent, ReactAgentConfig, MemoryLevel
from core.langchain_agent_tool import create_langchain_tool

# 设置工作目录
work_dir = "/home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/output/adaptive_mda"

# 清空工作目录
if os.path.exists(work_dir):
    shutil.rmtree(work_dir)
os.makedirs(work_dir)

print(f"工作目录: {work_dir}")

## 2. 配置LLM

可以选择不同的LLM来测试自适应分解

In [ ]:
# 选择LLM配置
# Gemini (能力强，通常不需要分解)
gemini_config = {
    "llm_model": "gemini-2.5-flash",
    "llm_base_url": "https://generativelanguage.googleapis.com/v1beta/openai/",
    "llm_api_key_env": "GEMINI_API_KEY",
    "llm_temperature": 0
}

# Kimi (能力中等，可能需要分解)
kimi_config = {
    "llm_model": "kimi-k2-turbo-preview",
    "llm_base_url": "https://api.moonshot.cn/v1",
    "llm_api_key_env": "MOONSHOT_API_KEY",
    "llm_temperature": 0
}

# 选择要使用的配置
llm_config = kimi_config  # 使用Kimi来演示自适应分解
print(f"使用LLM: {llm_config['llm_model']}")

## 3. 创建MDA任务TODO

定义每个步骤的后置断言

In [ ]:
# 创建MDA工作流TODO文件
mda_todo = """
# MDA工作流TODO

## 原始任务
从PIM生成完整的FastAPI应用并确保测试通过

## 步骤列表

### 步骤1：生成PSM
- [ ] 任务：根据PIM文件生成PSM
- 后置断言：blog_psm.md文件存在且包含5个核心章节
- 状态：待执行

### 步骤2：生成代码
- [ ] 任务：根据PSM生成FastAPI代码
- 后置断言：app/main.py存在，所有路由文件生成
- 状态：待执行

### 步骤3：生成测试
- [ ] 任务：生成单元测试
- 后置断言：tests/目录存在，至少3个测试文件
- 状态：待执行

### 步骤4：验证测试
- [ ] 任务：运行pytest验证
- 后置断言：pytest返回"0 failed"
- 状态：待执行

### 步骤5：修复失败（如需要）
- [ ] 任务：修复测试失败
- 后置断言：所有测试通过
- 状态：待执行

### 步骤6：运行应用
- [ ] 任务：启动FastAPI应用
- 后置断言：8000端口响应
- 状态：待执行
"""

# 保存TODO文件
todo_file = os.path.join(work_dir, "mda_workflow_todo.md")
with open(todo_file, 'w', encoding='utf-8') as f:
    f.write(mda_todo)

print(f"TODO文件已创建: {todo_file}")

## 4. 创建带后置断言验证的Agent

In [ ]:
def create_agent_with_postcondition(name, knowledge_files, interface, postcondition):
    """创建带后置断言的Agent"""
    config = ReactAgentConfig(
        work_dir=work_dir,
        memory_level=MemoryLevel.SMART,
        knowledge_files=knowledge_files,
        enable_project_exploration=False,
        **llm_config
    )
    
    agent = GenericReactAgent(config, name=name)
    agent.interface = interface
    # 将后置断言存储在单独的字典中，而不是作为属性
    # agent.postcondition = postcondition  # 这行会导致错误
    
    # 返回agent和postcondition作为元组
    return agent, postcondition

# PSM生成Agent
psm_agent, psm_postcondition = create_agent_with_postcondition(
    name="psm_generation_agent",
    knowledge_files=["knowledge/mda/pim_to_psm_knowledge.md"],
    interface="""PSM生成专家 - 根据PIM生成平台特定模型
    
# 重要：参考adaptive_task_decomposition.md
如果输出被截断或不完整，需要分解任务
""",
    postcondition="blog_psm.md存在且包含Domain Models, Service Layer, REST API Design, Application Configuration, Testing Specifications章节"
)

# 代码生成Agent
generation_agent, generation_postcondition = create_agent_with_postcondition(
    name="generation_agent",
    knowledge_files=["knowledge/mda/generation_knowledge.md"],
    interface="""代码生成专家 - 生成FastAPI代码
    
# 重要：如果代码生成不完整
1. 创建TODO列表分解任务
2. 逐个生成每个模块
3. 更新TODO状态
""",
    postcondition="app/main.py存在，app/models.py存在，app/schemas.py存在，tests/目录存在"
)

# 存储后置断言的字典
agent_postconditions = {
    "psm_generation_agent": psm_postcondition,
    "generation_agent": generation_postcondition
}

print("Agents创建完成，每个都有后置断言")

## 5. 执行任务并验证后置断言

In [ ]:
def execute_with_postcondition_check(agent, postcondition, task, step_name):
    """执行任务并验证后置断言"""
    print(f"\n{'='*60}")
    print(f"执行: {step_name}")
    print(f"后置断言: {postcondition}")
    print(f"{'='*60}")
    
    # 更新TODO状态为执行中
    update_todo_status(step_name, "执行中")
    
    # 执行任务
    result = agent.execute_task(task)
    
    # 验证后置断言
    postcondition_met = verify_postcondition(postcondition)
    
    if postcondition_met:
        print(f"✅ 后置断言满足")
        update_todo_status(step_name, "完成")
        return True, result
    else:
        print(f"❌ 后置断言失败，需要分解任务")
        update_todo_status(step_name, "失败-需分解")
        return False, result

def verify_postcondition(condition):
    """验证后置断言（简化版）"""
    # 这里可以根据条件类型进行实际验证
    # 现在简化为检查文件存在
    if "存在" in condition:
        # 提取文件名
        import re
        files = re.findall(r'([\w/]+\.\w+|[\w/]+/)', condition)
        for file in files:
            file_path = os.path.join(work_dir, file)
            if not os.path.exists(file_path):
                return False
    return True

def update_todo_status(step_name, status):
    """更新TODO文件状态"""
    # 读取TODO文件
    with open(todo_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 更新状态
    lines = content.split('\n')
    for i, line in enumerate(lines):
        if step_name in line:
            # 找到状态行并更新
            for j in range(i, min(i+5, len(lines))):
                if '状态：' in lines[j]:
                    lines[j] = f"- 状态：{status}"
                    break
    
    # 保存更新
    with open(todo_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))
    
    print(f"📝 TODO更新: {step_name} -> {status}")

## 6. 执行步骤1：生成PSM

In [ ]:
pim_file = "/home/guci/aiProjects/mda/pim-compiler/examples/blog.md"

psm_task = f"""
# PIM文件
{pim_file}

# 任务
根据上面的PIM生成PSM，文件名是blog_psm.md

# 后置断言
- PSM文件包含5个核心章节
- 每个实体都有定义

# 如果失败
参考knowledge/adaptive_task_decomposition.md进行任务分解
"""

success, result = execute_with_postcondition_check(
    psm_agent,
    psm_postcondition,  # 传递后置断言
    psm_task,
    "步骤1：生成PSM"
)

if not success:
    print("\n需要分解PSM生成任务...")
    # 这里可以实现分解逻辑

## 7. 任务分解示例

当任务失败时，自动分解为子任务

In [ ]:
def decompose_failed_task(original_task, failure_reason):
    """分解失败的任务"""
    decomposition_task = f"""
# 失败的任务
{original_task}

# 失败原因
{failure_reason}

# 请根据adaptive_task_decomposition.md的指导：
1. 分析失败类型（输出截断/部分失败/质量问题）
2. 生成子任务列表，每个子任务包含：
   - 任务描述
   - 后置断言
3. 使用TodoWrite工具创建子任务TODO
"""
    
    # 创建分解Agent
    decomposition_agent = GenericReactAgent(
        ReactAgentConfig(
            work_dir=work_dir,
            memory_level=MemoryLevel.SMART,
            knowledge_files=["knowledge/adaptive_task_decomposition.md"],
            **llm_config
        ),
        name="decomposition_agent"
    )
    
    # 执行分解
    decomposition_result = decomposition_agent.execute_task(decomposition_task)
    
    return decomposition_result

# 示例：如果代码生成失败
if False:  # 将此改为True来测试分解
    print("\n代码生成失败，开始分解任务...")
    
    subtasks = decompose_failed_task(
        "生成完整的FastAPI应用",
        "输出被截断，只生成了部分代码"
    )
    
    print(f"\n分解结果:\n{subtasks}")

## 8. 完整的MDA工作流（带自适应分解）

In [ ]:
def run_mda_workflow_with_decomposition():
    """运行带自适应分解的MDA工作流"""
    
    # 定义工作流步骤
    workflow_steps = [
        {
            "name": "生成PSM",
            "agent": psm_agent,
            "postcondition": psm_postcondition,
            "task": psm_task
        },
        {
            "name": "生成代码",
            "agent": generation_agent,
            "postcondition": generation_postcondition,
            "task": "根据blog_psm.md生成FastAPI代码"
        },
        # 更多步骤...
    ]
    
    # 执行工作流
    for step in workflow_steps:
        print(f"\n执行: {step['name']}")
        
        # 尝试执行
        max_attempts = 3
        for attempt in range(max_attempts):
            success, result = execute_with_postcondition_check(
                step['agent'],
                step['postcondition'],  # 传递后置断言
                step['task'],
                step['name']
            )
            
            if success:
                break
            
            if attempt < max_attempts - 1:
                print(f"\n尝试 {attempt + 1} 失败，分解任务...")
                decompose_failed_task(step['task'], "后置断言未满足")
            else:
                print(f"\n任务 {step['name']} 失败，需要人工介入")
                return False
    
    print("\n✅ MDA工作流完成！")
    return True

# 运行工作流
# success = run_mda_workflow_with_decomposition()

## 9. 查看TODO进度

In [ ]:
# 显示当前TODO状态
with open(todo_file, 'r', encoding='utf-8') as f:
    print(f.read())

## 总结

这个notebook展示了如何将自适应任务分解知识应用到MDA工作流：

### 关键改进

1. **后置断言**：每个步骤都有明确的成功条件
2. **TODO管理**：任务进度显式可见
3. **失败处理**：自动分解失败的任务
4. **知识驱动**：通过adaptive_task_decomposition.md指导分解

### 优势

- **适应不同模型**：Kimi失败时自动分解，Gemini可能一次成功
- **可恢复性**：中断后可以从TODO继续
- **可观测性**：进度透明，易于调试
- **质量保证**：后置断言确保每步正确